In [1]:
import pandas as pd
import dask.dataframe as dd
import dask
import coiled
import dask.distributed

In [2]:
cluster = coiled.Cluster(name="powers-demo", n_workers=10)

⠴ Creating Cluster. This might take a few minutes...

In [3]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1185: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| dask        | 2021.07.1 | 2021.07.2 | 2021.07.2 |
| distributed | 2021.07.1 | 2021.07.2 | 2021.07.2 |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [7]:
def create_by_partition_freq(partition_freq, years):
    for year in years:
        ddf = dask.datasets.timeseries(
            start=f'{year}-01-01',
            end=f'{year}-12-31',
            freq='1s',
            partition_freq=partition_freq,
            seed=42,
        )
        ddf.to_parquet(f"s3://coiled-datasets/timeseries/{partition_freq}/parquet/{year}")
        ddf.to_csv(f"s3://coiled-datasets/timeseries/{partition_freq}/csv/{year}/*.csv")

In [8]:
create_by_partition_freq("7d", ["2000", "2001"])

In [9]:
create_by_partition_freq("1h", ["2000", "2001"])

In [10]:
create_by_partition_freq("7d", ["2002", "2003", "2004", "2005"])

In [14]:
def create_1y_files(years):
    for year in years:
        ddf = dask.datasets.timeseries(
            start=f'{year}-01-01',
            end=f'{year}-12-31',
            freq='1s',
            partition_freq="7d",
            seed=42,
        )
        ddf.repartition(1).to_parquet(f"s3://coiled-datasets/timeseries/1y/parquet/{year}")
        ddf.repartition(1).to_csv(f"s3://coiled-datasets/timeseries/1y/csv/{year}/*.csv")

In [15]:
create_1y_files([2006])

In [18]:
create_1y_files([2007])

CancelledError: list-2d0255d9-1ccc-42cd-8ff8-da565108b2f7

In [5]:
dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/*/*.parquet")

,id,name,x,y
npartitions=312,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
ddf = dask.datasets.timeseries(
    start='2000-01-01',
    end='2020-12-31',
    freq='1s',
    partition_freq='7d',
    seed=42,
)
ddf.to_parquet("s3://coiled-datasets/timeseries/20-years/parquet/")

In [7]:
len(dd.read_parquet("s3://coiled-datasets/timeseries/20-years/parquet/"))

662256000

In [4]:
ddf = dask.datasets.timeseries(
    start='2000-01-01',
    end='2020-12-31',
    freq='1s',
    partition_freq='7d',
    seed=42,
)
ddf.to_csv("s3://coiled-datasets/timeseries/20-years/csv/")

['coiled-datasets/timeseries/20-years/csv/0000.part',
 'coiled-datasets/timeseries/20-years/csv/0001.part',
 'coiled-datasets/timeseries/20-years/csv/0002.part',
 'coiled-datasets/timeseries/20-years/csv/0003.part',
 'coiled-datasets/timeseries/20-years/csv/0004.part',
 'coiled-datasets/timeseries/20-years/csv/0005.part',
 'coiled-datasets/timeseries/20-years/csv/0006.part',
 'coiled-datasets/timeseries/20-years/csv/0007.part',
 'coiled-datasets/timeseries/20-years/csv/0008.part',
 'coiled-datasets/timeseries/20-years/csv/0009.part',
 'coiled-datasets/timeseries/20-years/csv/0010.part',
 'coiled-datasets/timeseries/20-years/csv/0011.part',
 'coiled-datasets/timeseries/20-years/csv/0012.part',
 'coiled-datasets/timeseries/20-years/csv/0013.part',
 'coiled-datasets/timeseries/20-years/csv/0014.part',
 'coiled-datasets/timeseries/20-years/csv/0015.part',
 'coiled-datasets/timeseries/20-years/csv/0016.part',
 'coiled-datasets/timeseries/20-years/csv/0017.part',
 'coiled-datasets/timeseries

In [6]:
len(dd.read_csv("s3://coiled-datasets/timeseries/20-years/csv/*.part"))

662256000

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
